# Find Existing and Planned JWST Observations Using _astroquery_

## Introduction

As with _HST, JWST_ observers are not allowed to propose observations that duplicate existing, planned, or approved observations unless they provide a scientific justification in their proposal and that request is approved. Consult the [JWST Duplicate Observation Policy](https://jwst-docs.stsci.edu/jwst-opportunities-and-policies/jwst-general-science-policies/jwst-duplicate-observations-policy) for details. Broadly speaking, observations might duplicate if they are obtained with the same scientific instrument (or a different instrument with similar configurations and capabilities), and two or more of the following apply:
  * Same astrophysical source, or significant spatial overlap of fields
  * Similar imaging passband, or overlapping spectral range
  * Similar (spectral) resolution
  * Similar exposure depth

This notebook illustrates how to use the python package [astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html) to search the Mikulski Archive for Space Telescopes (MAST) for potential duplicate observations. Proposers may also use the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) to search the archive, but that may be less efficient for large numbers of targets. 

<ul>
    <li><a href="#Setup">Setup</a></li>
    <li><a href="#Example-Queries">Example Queries</a></li>
    <ul>
        <li><a href="#Target-by-Name">Single Target by Name</a></li>
        <li><a href="#Moving-Target">Single Moving Target</a></li>
        <li><a href="#Target-Field">Target Field by Position</a></li>
        <li><a href="#Target-List">Search a Target List</a></li>
        <li><a href="#upload-targets">Loading Targets from a File</a></li>
    </ul>
    <li><a href="#Resources">Additional Resources</a></li>
</ul>

### Special Disclaimer
<a id="Disclaimer"></a>

The capabilities described here will help <em>identify</em> potential duplications between your intended JWST observations and those that have been approved, planned, or that have already executed. 

<div class="alert alert-block alert-info">

<span style="color:black">
The complete footprint of approved (but not executed) dithered or mosaicked observations is only approximate. That is, only the primary location is reported for an observation, but not necessarily those for associated dither positions or mosaic tiles. Moreover metadata in MAST about planned/approved observations is <b>not sufficient</b> to determine precisely whether your intended observation is a genuine duplication, particularly for slit or MOS spectroscopy. You are responsible for evaluating the details of the planned observations by using the accepted program's APT file (and/or the Aladin display in APT, as appropriate) to determine if the potential duplications are genuine.
</span>
</div>


<a id="Setup"></a>
## Setup

We begin by importing some essential python packages: general utilities in [astropy](https://www.astropy.org/), the [pandas](https://pandas.pydata.org/) data manipulation library, and query services in astroquery. We also define a utility routine to create URLs to the parent programs of matching observations. 

In [ ]:
import astropy
from astropy import table
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import Angle
from astroquery.mast import Mast
from astroquery.mast import Observations
import numpy as np

# Give the notebook cells more of the available width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

APT_LINK = 'http://www.stsci.edu/cgi-bin/get-proposal-info?id={}&observatory=JWST'

def get_program_URL(program_id):
    """
    Generate the URL for program status information, given a program ID. 
    """
    return APT_LINK.format(program_id)

The results of an astroquery search are contained in an [astropy table](https://docs.astropy.org/en/stable/table/). There are multiple ways to display the results; the function below displays table fields that are most relevant for identifying potential duplications of JWST observations, and should be treated as illustrative. 

In [ ]:
def display_results(obs):
    """
    Simple display of results related to identifying potentially duplicating targets.
    Observation program title is truncated for presentation in this notebook
    """
    # build the URL to the JWST programs.
    obs['proposal_URL'] = [get_program_URL(x) for x in obs['proposal_id']]
    obs['obs_title'] = [x[:70] for x in obs['obs_title']]
    obs['obs_title'].info.format = '<'
    obs['target_name', 'instrument_name', 'filters', 'dataproduct_type', 't_exptime', 
        'proposal_id'].pprint(max_lines=40, max_width=90)
    
    print("\nUnique Program Titles:")
    table.unique(obs, keys=['proposal_id'])['proposal_id','obs_title'].pprint(max_width=100)
    print("\nUnique URLs to status of existing programs:")
    for i in set(obs['proposal_URL']):
        print(i)

<a id="Example-Queries"></a>
## Example Queries

All of the queries below search for JWST observations, using a search radius somewhat larger than fields of view (FoV) of interest, to allow for the possibility that the FoV may be rotated when approved-but-unexecuted observations are actually scheduled. If your intended observation uses a different FoV, then adjust the search radius accordingly. 

<a id="Target-by-Name"></a>
### Single Target by Name

This example shows how to query for a single target with a standard name, **Trappist-1** which is a star with a known exo-planet. The intended observations would be timeseries imaging in a small FoV. Note that the name will be resolved automatically to coordinates in this case. We use the <b><code>query_criteria()</code></b> method to limit the search to JWST observations. 

In [ ]:
obs = Observations.query_criteria(
        objectname="Trappist-1", 
        radius="10s", 
        obs_collection="JWST"
        )
print('Number of matching observations: {}'.format(len(obs)))

Examine the returned table columns most relevant for identifying potential duplications. Note: it is still up to you to determine if these observations count as a duplicate with those you were planning. For instance, it does not provide the timing information necessary to determine which TRAPPIST-1 planet they are targetting. In some cases, the target name or proposal title (<code>obs_title</code>) contains this information.

In [ ]:
display_results(obs)

<a id="Moving-Target"></a>
### Single Moving Target

This example shows how to query for a moving target. This kind of search is limited to a modest set of solar system bodies with recognized names. Note the use of a wildcard character (*) in case the target name includes other text.

In [ ]:
obs = Observations.query_criteria(
        target_name="Io*",
        obs_collection="JWST"
        )
    
display_results(obs)

<a id="Target-Field"></a>
### Target Field by Position

This example shows how to search an area of sky for overlap with a proposed deep field. The field center (RA, Dec) is (12:12:22.513, +27:34:13.88), and the planned survey area is 30&times;30 arcmin. We will limit the search to JWST imaging observations. First, convert the coordinate representationn to degrees, then execute the search.

In [ ]:
ra_deg = Angle('12:12:22.513 hours').degree
dec_deg = Angle('+27:34:13.88 degree').degree
obs = Observations.query_criteria(
        s_ra=[ra_deg-0.25,ra_deg+0.25],
        s_dec=[dec_deg-0.25,dec_deg+0.25],
        dataproduct_type="image",
        obs_collection="JWST"
        )

There is clearly an overlap with another program, but only in certain filters:

In [ ]:
display_results(obs)

<a id="Target-List"></a>
### Search for Observations of Targets in a List

It may be best to search for individual targets (as above) with the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) because the results are easily visualized. But it may be more efficient to search for a large list of targets using astroquery. 

Your list might be stored in a file on your local system, and consist of coordinates and custom search radii. But for simplicity the list in this example consists of standard target names, constructed in code. Not all of the targets have approved or existing JWST observations, so the first step is to determine the number of observations for each target using the astroquery method <b><code>Observations.query_criteria_count()</code></b>.

<div class="alert alert-block alert-info">

<span style="color:black">
    It is good practice to first check the number of matching observations before fetching the results themselves, in case the number of results is extremely large. This is more important when querying large MAST missions, such as <i>HST</i>. Note that even for a modest number of results this query may take several seconds.
    
</span>
</div>

In [ ]:
# Create a dict to contain the number of observations for each target, initialized to zero
targets = {name:0 for name in ['CX Tau','Fomalhaut','HL Tauri','M 8','30 Dor']}

search_radius = '30s'
for t,n in targets.items():
    targets[t] = Observations.query_criteria_count(
            objectname=t, 
            radius='{}'.format(search_radius), 
            obs_collection='JWST'
            )

targets

It is clear that none of the targets in the list has an excessive number of matching observations. Now check the results for the targets with non-zero matching observations in detail. <b>Note:</b> since the loop creates one astropy table for each search, we place each in a list and then concatenate them for display. 

In [ ]:
obs_list = []
for t,n in targets.items():
    if n > 0:
        obs = Observations.query_criteria(
            objectname=t, 
            radius='{}'.format(search_radius), 
            obs_collection='JWST'
            )
        obs_list.append(obs)
        
target_matches = table.vstack(obs_list)
display_results(target_matches)

If you write the results table to a disk file in ECSV format (see [astropy table I/O](https://docs.astropy.org/en/stable/io/unified.html#table-io-ascii)), that will preserve the table metadata, as well as the option for reading the file as an astropy table in a subsequent python session.

In [ ]:
target_matches.write('target_matches.ecsv', format='ascii.ecsv', overwrite=True)

<a id="upload-targets"></a>
### Loading Targets from a File
It may be more efficient to read in a list of targets from a local file rather than manually specifying each one. You can load an example file of targets, `targets.csv` on disk using an astropy function. The file contains:
```
target_name, RA, DEC
CX Tau, 4:14:47.861, +26:48:10.91
Fomalhaut, 22:57:39.046, -29:37:20.05
HL Tau, 0:31:38.437, +18:13:57.65
M 8, 18:03:36.960, -24:23:13.20
30 Dor, 5:38:42.396, -69:06:03.36
```
The first row of the file will be interpreted as a column name in the table. <b>This is important.</b> 

In [ ]:
# load objects as target coordinates
targets = Table.read('targets.csv', format='ascii.csv')
targets

With your targets in memory, you may query on them using the same ``astroquery`` functions as described above.  For example, query by source identifier using of target names to get the count of the number of results before performing the query for observations in MAST.

<div class="alert alert-block alert-info">

<span style="color:black">
    Query by name is not always possible: it depends on whether the name is common (can be recognized as an astrophysical source in, e.g., the Vizier catalog). You may need to query by coordinates instead.
    
</span>
</div>

In [ ]:
# convert names to dictionary to hold result counts
target_names = {name:0 for name in targets['target_name']}

# get the counts of each target
search_radius = '30s'
for t,n in target_names.items():
    target_names[t] = Observations.query_criteria_count(
            objectname=t, 
            radius='{}'.format(search_radius), 
            dataproduct_type="spectrum",
            obs_collection='JWST'
            )

targets['N_obs'] = list(target_names.values())

The file contains equitorial coordinates in sexagesimal format, which need to be converted to degrees if searching on a region of sky rather than by object name. For this we use the astroquery Angle class.

In [ ]:
targets['ra_deg'] = [Angle(x+' hours').degree for x in targets['RA']]
targets['dec_deg'] = [Angle(x+' degree').degree for x in targets['DEC']]

This example will use a constant search window (i.e., spatial extent on the sky) of 0.1&deg;, but it would be simple to customize the search area per target. This search will be for <b><i>spectrum data products only</i></b>, to illustrate the case where images are not important for identifying duplications. Each target search returns a separate object table, so we concatenate them for display. Note: we only search for targets where the count of observations is greater than zero to save a little time.

In [ ]:
window = 0.05
obs_list = []
for r in targets:
    if r['N_obs'] > 0:
        ra_range = [r['ra_deg']-window, r['ra_deg']+window]
        dec_range = [r['dec_deg']-window, r['dec_deg']+window]
        obs = Observations.query_criteria(
            s_ra=ra_range,
            s_dec=dec_range,
            dataproduct_type="spectrum",
            obs_collection="JWST"
            )
        obs_list.append(obs)
    
target_matches = table.vstack(obs_list)
display_results(target_matches)

<a id="Resources"></a>
# Additional Resources

* [astropy](https://docs.astropy.org/en/stable/index.html) documentation
* [astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html) documentation for querying MAST
* [Queryable fields](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html) in the MAST/CAOM database
* The [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) web interface

## About this notebook

This notebook was developed by Archive Sciences Branch staff, chiefly Susan Mullally and Dick Shaw. For support, please contact the Archive HelpDesk at archive@stsci.edu, or through the [JWST HelpDesk Portal](https://jwsthelp.stsci.edu). 
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>